In [97]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec

# Support Vector Classifier Algorithm
from sklearn.svm import SVC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import transform_series_to_dictionary

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [3]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Type of measure']

## word2vec model

In [4]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
model = Word2Vec.load('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')
w2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [5]:
class MeanEmbeddingVectorizer(object):
    """Calculate the mean of each word"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(w2v_dict))])
        else:
            self.dim=0

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0) for words in X]
        )


In [90]:
# Use pipes to implement steps of fit and transform method
svm_w2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_dict)),
    # Use SVC algorithm
    # :gamma: is a parameter for non linear hyperplanes. The higher the gamma
    #         value it tries to exactly fit the training data set.
    #
    # :C: is the penalty parameter of the error term. It controls the trade
    #     off between smooth decision boundary and classifying the training points correctly.
    #     !!! Increasing C values may lead to overfitting the training data. !!!
    ("SVM", SVC(gamma=40, C=100))])

## Train SVM Model

In [91]:
# Split data into test and train sets
# Where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=42, test_size=0.3, shuffle=True)

In [92]:
# Fit SVM model with out train data
svm_w2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <__main__.MeanEmbeddingVectorizer object at 0x7f5850127ee0>),
                ('SVM', SVC(C=100, gamma=40))])

In [93]:
# Model Evaluation
svm_w2v.score(X_train, y_train)

0.9801223241590215

In [94]:
# SVM prediction based on test data
prediction = svm_w2v.predict(X_test)
prediction

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0,
       0, 0, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 5,
       2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 0, 0, 3, 0, 1, 0, 2, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 2, 0, 0, 0, 1, 0, 2, 0, 1, 4, 0, 0, 0, 0, 2, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

## Evaluation

In [98]:
# View evaluation metrics table
evaluation = model_evaluation_metrics(y_test, prediction, 'Type of measure')

In [100]:
# Transform columns data to dict to use in MLFlow
transformation = transform_series_to_dictionary(evaluation, 'Evaluation Metrics', 'Type of measure')

In [101]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Type of measure (Word2Vec)")

with mlflow.start_run():
    parameters = {"Language model": "word2vec",
                  "Gamma": 40,
                  "C": 100,
                  "Random state": 42,
                  "Test size": 0.3}
    log_params(parameters)

    log_metrics(transformation)